# Carga de datos

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [172]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ruta = '/content/Tablas_Optimiazacion_Estocastica.xlsx'

hojas = {
    "Cliente": "Clientes",
    "Producto": "Productos",
    "Máquina": "Máquinas",
    "Tasas_Producción": "Rates"
}
dataframes = {nombre: pd.read_excel(ruta, sheet_name=hoja) for nombre, hoja in hojas.items()}

df_clientes = dataframes["Cliente"]
df_productos = dataframes["Producto"]
df_maquinas = dataframes["Máquina"]
df_tasas_produccion = dataframes["Tasas_Producción"]


# Clases

## Cliente

In [346]:
class Cliente:
    def __init__(self, id_cliente, nivel_cliente, tiempo_cliente, probabilidad_compra):
        """
        Representa un cliente con características adicionales.
        :param id_cliente: Identificador único del cliente.
        :param nivel_cliente: Nivel del cliente (valor continuo entre 0 y 1).
        :param tiempo_cliente: Tiempo de relación con el cliente (valor continuo entre 0 y 1).
        """
        self.id_cliente = id_cliente
        self.nivel_cliente = nivel_cliente
        self.tiempo_cliente = tiempo_cliente
        self.probabilidad_compra = probabilidad_compra


## Producto

In [348]:
class Producto:
    def __init__(self, id_producto, producto, subproductos, nombre_subproducto,
                 tamanio, dificultad, precio, popularidad, demanda_promedio, demanda_desviacion):
        """
        Representa un producto y sus subproductos.
        :param id_producto: Identificador único del producto.
        :param producto: Nombre del producto.
        :param subproductos: Lista de subproductos asociados (ID_Subproducto, Nombre, Tamaño).
        :param dificultad: Dificultad asociada (1-5).
        :param precio: Precio del producto (60-130).
        :param popularidad: Popularidad del producto (0-1).
        :param demanda_promedio: Demanda promedio del producto.
        :param demanda_desviacion: Desviación estándar de la demanda del producto.
        """
        self.id_producto = id_producto
        self.producto = producto
        self.subproductos = subproductos
        self.dificultad = dificultad
        self.precio = precio
        self.tamanio = tamanio
        self.nombre_subproducto = nombre_subproducto
        self.popularidad = popularidad
        self.demanda_promedio = demanda_promedio
        self.demanda_desviacion = demanda_desviacion


## Máquina

In [262]:
class Maquina:
    def __init__(self, id_maquina, nombre, procesos, tiempo_ocupado = 0, estado = 'on'):
        """
        Representa una máquina en el sistema de producción.
        :param id_maquina: Identificador único de la máquina.
        :param nombre: Nombre descriptivo de la máquina.
        :param procesos: Tipo de procesos que realiza (e.g., Semiterminado, Envasado).
        :param tasa_produccion: Tasa de producción (Unidades/secuencia).
        :param tiempo_preparacion: Tiempo de preparación (min).
        :param tiempo_limpieza: Tiempo de limpieza (min).
        """
        self.id_maquina = id_maquina
        self.nombre = nombre
        self.procesos = procesos
        self.estado = estado
        self.turnos = []
        self.tiempo_ocupado = tiempo_ocupado

    def asignar_turno(self, turno):
        self.turnos.append(turno)
        self.tiempo_ocupado += turno['duracion']

    def tiene_espacio(self, duracion):
        JORNADA_TOTAL = 480
        """Verifica si la máquina tiene suficiente espacio en su jornada para una tarea."""
        return self.tiempo_ocupado + duracion <= JORNADA_TOTAL

    def __str__(self):
        return f"Máquina {self.id_maquina}, Estado: {self.estado}, Turnos: {self.turnos}, Tiempo ocupado: {self.tiempo_ocupado}"


## Rate

In [176]:
class Rate:
    def __init__(self, id_maquina, id_producto, tasa_produccion, tiempo_preparacion, tiempo_limpieza, tiempo_total):
        """
        Clase para modelar las tasas de producción de las máquinas.

        :param id_maquina: ID de la máquina.
        :param id_producto: ID del producto asociado.
        :param tasa_produccion: Unidades producidas por secuencia.
        :param tiempo_preparacion: Tiempo de preparación en minutos.
        :param tiempo_limpieza: Tiempo de limpieza en minutos.
        :param tiempo_total: Tiempo total del proceso (preparación + limpieza + producción) en minutos.
        """
        self.id_maquina = id_maquina
        self.id_producto = id_producto
        self.tasa_produccion = tasa_produccion
        self.tiempo_preparacion = tiempo_preparacion
        self.tiempo_limpieza = tiempo_limpieza
        self.tiempo_total = tiempo_total

## Pedido

In [362]:
class Pedido:
    def __init__(self, id_pedido, id_cliente, id_producto, demanda, valor):
        """
        Representa un pedido realizado por un cliente.
        :param id_pedido: Identificador único del pedido.
        :param id_cliente: ID del cliente asociado.
        :param id_producto: ID del producto solicitado.
        :param demanda: Cantidad demandada del producto.
        :param valor: Valor del pedido (normalizado o calculado).
        """
        self.id_pedido = id_pedido
        self.id_cliente = id_cliente
        self.id_producto = id_producto
        self.demanda = demanda
        self.valor = valor

## Secuencias

In [178]:
class Secuencia:
    def __init__(self, id_secuencia, id_pedido, id_maquina, id_producto, proceso, inicio, fin, dia=1):
        """
        Representa una secuencia específica en el sistema de producción.
        :param id_secuencia: Identificador único de la secuencia.
        :param id_pedido: Identificador del pedido asociado.
        :param id_maquina: Identificador de la máquina usada.
        :param id_producto: Identificador del producto procesado.
        :param proceso: Descripción del proceso (e.g., "Hacer el producto").
        :param inicio: Hora de inicio en formato minutos (0-1440).
        :param fin: Hora de fin en formato minutos (0-1440).
        :param dia: Día en que se ejecuta la secuencia.
        """
        self.id_secuencia = id_secuencia
        self.id_pedido = id_pedido
        self.id_maquina = id_maquina
        self.id_producto = id_producto
        self.proceso = proceso
        self.inicio = inicio
        self.fin = fin
        self.dia = dia


# Creación de objetos

In [419]:
clientes = [Cliente(row['ID_cliente'], row['Nivel_Cliente'], row['Tiempo_Cliente'], 0.1)
            for _, row in df_clientes.iterrows()]

productos = [Producto(row['ID_producto'], row['Producto'], row['ID_Subproducto'],
                      row['Nombre_Subproducto'], row['Tamaño (mL)'], row['Dificultad'],
                      row['Precio ($ mxn)'], 0.2, 160, 20)
             for _, row in df_productos.iterrows()]

maquinas = [Maquina(row['ID_Maquina'], row['Nombre'], row['Procesos'])
            for _, row in df_maquinas.iterrows()]

rates = [
    Rate(
        id_maquina=row['Máquina'],
        id_producto=row['ID Producto'],
        tasa_produccion=row['Tasa Producción (Unidades/secuencia)'],
        tiempo_preparacion=row['Tiempo_Preparación (min)'],
        tiempo_limpieza=row['Tiempo_Limpieza (min)'],
        tiempo_total=row['Tiempo_Total (min)']
    )
    for _, row in df_tasas_produccion.iterrows()
]

# Generación de pedidos

In [180]:
def validarParametros(parametros):
    for parametro in parametros:
        if parametro < 0:
            return "Error: Todos los parámetros deben ser mayores o iguales a 0"
        elif parametro > 10:
            return "Error: Los parámetros no pueden ser mayores a 10"
        elif not isinstance(parametro, int):
            return "Error: Los parámetros deben ser números enteros"
    return True


In [186]:
def valor_de_pedido(producto_pedido, cliente, demanda, parametros, cantidad_total):
    id_cliente = cliente.id_cliente
    nivel_cliente = cliente.nivel_cliente
    tiempo_cliente = cliente.tiempo_cliente

    dificultad = producto_pedido.dificultad
    precio = producto_pedido.precio
    cantidad_norm = demanda / cantidad_total

    # Fórmula del valor prioridad
    valor = (
        10 * parametros[0] * (parametros[2] * nivel_cliente + parametros[3] * tiempo_cliente) +
        parametros[1] * (parametros[4] * cantidad_norm + 2 * parametros[5] * precio + parametros[6] / 2 * (1 - abs(dificultad)))
    )
    #print(f'Valor del pedido: {valor}, Nivel de cliente: {nivel_cliente}, Tiempo del cliente: {tiempo_cliente}, Dificultad: {dificultad}, Precio: {precio}, Cantidad normalizada: {cantidad_norm}')
    return valor


In [215]:
def generar_pedido(clientes, productos, num_pedidos, parametros, max_demand=500):
    """
    Genera un pedido aleatorio para un cliente, con una cantidad aleatoria de productos demandados.
    """
    pedidos = []
    i = 0
    cantidad_productos = [40,80,120,160,200,240,280,320,360,400]
    demandas = [random.choice(cantidad_productos) for _ in range(num_pedidos)]
    #demandas = [random.randint(1, max_demand) for _ in range(num_pedidos)]
    cant_total = sum(demandas)
    for i in range(num_pedidos):
        cliente = random.choice(clientes)
        demanda = demandas[i]
        producto_pedido = random.choice(productos)

        valor = valor_de_pedido(producto_pedido, cliente, demanda, parametros, cant_total)
        pedido = Pedido(f'W{i+1}', cliente.id_cliente, producto_pedido.id_producto, demanda, valor)
        #print(f'Pedido: {pedido.id_pedido}, Cliente: {pedido.id_cliente}, Producto: {pedido.id_producto}, Demanda: {pedido.demanda}, Valor: {pedido.valor}')
        pedidos.append(pedido)

    pedidos.sort(key=lambda p: p.valor, reverse=True)
    for pedido in pedidos:
        print(f'Pedido: {pedido.id_pedido}, Cliente: {pedido.id_cliente}, Producto: {pedido.id_producto}, Demanda: {pedido.demanda}, Valor: {pedido.valor}')

    return pedidos

In [232]:
# Parámetros ajustables
beta1 = 1 #Entero: peso de la función de valor de cliente
beta2 = 1 #Entero: peso de la función de valor de producto
alpha1 = 1 #Entero: peso del nivel del cliente
alpha2 = 1 #Entero: peso del tiempo del cliente
lambda1 = 1 #Entero: peso de la cantidad de producto
lambda2 = 1 #Entero: peso del precio del producto
lambda3 = 1 #Entero: peso de la dificultad del producto

parametros = [beta1, beta2, alpha1, alpha2, lambda1, lambda2, lambda3]

if validarParametros(parametros) == True:
    pedidos = generar_pedido(clientes, productos, 3, parametros, max_demand=100)

Pedido: W2, Cliente: C6, Producto: P6, Demanda: 400, Valor: 265.2871198766591
Pedido: W1, Cliente: C5, Producto: P5, Demanda: 200, Valor: 171.5963160825191
Pedido: W3, Cliente: C2, Producto: P3, Demanda: 80, Valor: 104.61351093410653


# Simulación Monte Carlo

In [ ]:
def actualizar_probabilidades(clientes, productos, pedidos_realizados, iteracion, intervalo_actualizacion=10):
    """
    Actualiza las probabilidades de compra y popularidad de los productos basados en los pedidos realizados.
    """
    if iteracion % intervalo_actualizacion == 0:
        # Actualizar probabilidad de compra de los clientes basados en la demanda total generada
        for cliente in clientes:
            total_compras = sum([pedido.demanda for pedido in pedidos_realizados if pedido.id_cliente == cliente.id_cliente])
            cliente.probabilidad_compra = min(1, cliente.probabilidad_compra + (total_compras / 1000))

        # Actualizar popularidad de los productos basado en la cantidad vendida
        for producto in productos:
            total_vendido = sum([pedido.demanda for pedido in pedidos_realizados if pedido.id_producto == producto.id_producto])
            producto.popularidad = min(1, producto.popularidad + (total_vendido / 1000))

In [426]:
def simular_pedidos(clientes, productos, num_pedidos, parametros, iteraciones=100, max_demand=500):
    """
    Simula una serie de pedidos generados por los clientes, ajustando las probabilidades de compra y popularidad
    de los productos durante la simulación.
    """
    pedidos_acumulados = []
    for i in range(iteraciones):
        pedidos_realizados = []
        for _ in range(num_pedidos):
            cliente = random.choices(clientes, weights=[cliente.probabilidad_compra for cliente in clientes])[0]
            producto = random.choices(productos, weights=[producto.popularidad for producto in productos])[0]

            demanda = random.randint(1, max_demand)
            valor = valor_de_pedido(producto, cliente, demanda, parametros, max_demand)

            pedido = Pedido(f'W{i+1}', cliente.id_cliente, producto.id_producto, demanda, valor)
            pedidos_realizados.append(pedido)

        pedidos_acumulados.extend(pedidos_realizados)
        actualizar_probabilidades(clientes, productos, pedidos_acumulados, i)
        if i % 100 == 0:
            print(f'Pedido: {pedido.id_pedido}, Cliente: {pedido.id_cliente}, Producto: {pedido.id_producto}, Demanda: {pedido.demanda}, Valor: {pedido.valor}')

    pedidos_acumulados.sort(key=lambda p: p.valor, reverse=True)
    return pedidos_acumulados

parametros = {
    'max_demand': 500
}

iteraciones = 1000
num_pedidos = 10

simulacion_pedidos = simular_pedidos(clientes, productos, num_pedidos, parametros, iteraciones)


Pedido: W1, Cliente: C7, Producto: P7, Demanda: 155, Valor: 6.780033829803192
Pedido: W101, Cliente: C9, Producto: P2, Demanda: 154, Valor: 23.50602195249531
Pedido: W201, Cliente: C5, Producto: P4, Demanda: 120, Valor: 20.531083733472755
Pedido: W301, Cliente: C9, Producto: P3, Demanda: 314, Valor: 23.96393147105041
Pedido: W401, Cliente: C1, Producto: P3, Demanda: 197, Valor: 15.681038826297202
Pedido: W501, Cliente: C2, Producto: P1, Demanda: 84, Valor: 4.769120577227167
Pedido: W601, Cliente: C5, Producto: P4, Demanda: 115, Valor: 19.675621911244722
Pedido: W701, Cliente: C9, Producto: P4, Demanda: 14, Valor: 1.7095288692723862
Pedido: W801, Cliente: C5, Producto: P3, Demanda: 287, Valor: 30.68969287243063
Pedido: W901, Cliente: C10, Producto: P1, Demanda: 397, Valor: 0.7683707248636157


In [431]:
def generar_pedido_ponderado(clientes, productos, num_pedidos, parametros, max_demand=500):
    """
    Genera un pedido aleatorio para un cliente, con una cantidad aleatoria de productos demandados.
    Esta versión usa distribuciones ponderadas por probabilidad de compra y popularidad.
    """
    pedidos = []
    cantidad_productos = [40, 80, 120, 160, 200, 240, 280, 320, 360, 400]
    demandas = [random.choice(cantidad_productos) for _ in range(num_pedidos)]
    cant_total = sum(demandas)

    for i in range(num_pedidos):
        cliente = random.choices(clientes, weights=[cliente.probabilidad_compra for cliente in clientes])[0]

        producto = random.choices(productos, weights=[producto.popularidad for producto in productos])[0]

        demanda = max(0, int(random.gauss(producto.demanda_promedio, producto.demanda_desviacion)))

        demanda = min(demanda, max_demand)
        valor = valor_de_pedido(producto, cliente, demanda, parametros, cant_total)

        pedido = Pedido(f'W{i+1}', cliente.id_cliente, producto.id_producto, demanda, valor)

        pedidos.append(pedido)

    pedidos.sort(key=lambda p: p.valor, reverse=True)

    for pedido in pedidos:
        print(f'Pedido: {pedido.id_pedido}, Cliente: {pedido.id_cliente}, Producto: {pedido.id_producto}, Demanda: {pedido.demanda}, Valor: {pedido.valor}')

    return pedidos

In [433]:
pedidos_ponderados = generar_pedido_ponderado(clientes, productos, 10, parametros)

Pedido: W10, Cliente: C1, Producto: P6, Demanda: 153, Valor: 9.423974566565487
Pedido: W3, Cliente: C7, Producto: P6, Demanda: 170, Valor: 7.1927202207493535
Pedido: W7, Cliente: C1, Producto: P5, Demanda: 165, Valor: 6.2542214318081655
Pedido: W1, Cliente: C2, Producto: P6, Demanda: 159, Valor: 5.821152348550026
Pedido: W5, Cliente: C1, Producto: P1, Demanda: 155, Valor: 4.406383281501208
Pedido: W6, Cliente: C7, Producto: P1, Demanda: 202, Valor: 3.9446049355421797
Pedido: W9, Cliente: C8, Producto: P2, Demanda: 154, Valor: 3.7439251707626617
Pedido: W8, Cliente: C2, Producto: P1, Demanda: 168, Valor: 2.8387622483495045
Pedido: W4, Cliente: C7, Producto: P3, Demanda: 167, Valor: 2.717611486120231
Pedido: W2, Cliente: C3, Producto: P2, Demanda: 191, Valor: 0.4866266435415214
